In [1]:
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LSTM, Embedding

In [11]:
# Load pre-trained VGG16 model without the top layer
model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the layers of the VGG16 model
for layer in model.layers:
    layer.trainable = False

# Add a custom layer to extract features from the VGG16 model
x = model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
features = Dense(64, activation='relu')(x)

# Build a LSTM model to generate hashtags based on the extracted features
lstm_input = keras.Input(shape=(64,))
lstm_output = Embedding(input_dim=64, output_dim=32, input_length=64)(lstm_input)
lstm_output = LSTM(32, return_sequences=False)(lstm_output)
lstm_output = Dense(64, activation='relu')(lstm_output)
lstm_output = Dense(32, activation='relu')(lstm_output)
lstm_output = Dense(16, activation='softmax')(lstm_output)

# Combine the VGG16 model and the LSTM model
model = Model(inputs=model.input, outputs=lstm_output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

# Evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))


['#happy', '#cute', '#selfie']
